In [1]:
import itertools
import random
import requests
import time
from math import sin, cos, radians, atan2, sqrt
from typing import NamedTuple
import secrets

URL = 'https://api.acro.bike/cross/ab_mapp'

class Point(NamedTuple):
    lat: float
    lng: float
    
    def dist(self, p: "Point"):
        R = 6373000.0

        lat1 = radians(p.lat)
        lng1 = radians(p.lng)
        lat2 = radians(self.lat)
        lng2 = radians(self.lng)

        deltaLng = lng2 - lng1
        deltaLat = lat2 - lat1

        a = sin(deltaLat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(deltaLng / 2) **2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        return distance
    
    def move(self, deltaLat, deltaLng):
        return Point(lat=self.lat + deltaLat, lng=self.lng + deltaLng)

CENTER = Point(lat=52.217049, lng=21.017532)
RANGE_LAT = range(-10, 11)
RANGE_LNG = range(-10, 11)
DELTA_LAT = 0.01
DELTA_LNG = 0.01
DELTAS = itertools.product(RANGE_LAT, RANGE_LNG)
POINTS = list(map(lambda x: CENTER.move(x[0] * DELTA_LAT, x[1] * DELTA_LNG), DELTAS))

def login():
    sess = requests.Session()
    data = {
        "method": "UserLogin",
        "params": {
            "phone": secrets.PHONE,
            "devicenum": secrets.DEVICENUM
        }
    }
    getJSession = sess.get(URL)
    loginRequest = sess.post(URL, json=data, cookies=getJSession.cookies)
    print(loginRequest.json())
    return sess

def getBikes(sess, point):
    data = {
        "method": "GetBike",
        "params": {
            "longitude": str(point.lng),
            "latitude": str(point.lat),
            "timestamp": int(time.time())
        }
    }
    bikeRequest = sess.post(URL, json=data)
    result = bikeRequest.json()['info']['aroundbikes']
    return result

def getMultipleBikes(sess, points):
    result = []
    for point in points:
        bikes = getBikes(sess, point)
        #time.sleep(random.random())
        for bike in bikes:
            dup = list(filter(lambda x: x['bikeid'] == bike['bikeid'], result))
            if len(dup) > 0:
                result.remove(dup[0])
            result.append(bike)
    return result
    

In [5]:
sess = login()

bikes = getMultipleBikes(sess, POINTS)
print(len(bikes))

{'result': 0, 'info': {'identitycheck': '1', 'hasic': 0, 'balance': '2.00', 'voucher': '0.00', 'depositFree': False, 'userrole': '0', 'foregift': 49.0, 'pushType': '0'}}
207


In [3]:
sess = login()

bikes = getBikes(sess, CENTER)
print(len(bikes))

{'result': 0, 'info': {'identitycheck': '1', 'hasic': 0, 'balance': '2.00', 'voucher': '0.00', 'depositFree': False, 'userrole': '0', 'foregift': 49.0, 'pushType': '0'}}
76


In [6]:
import json
from pathlib import Path

def latLng(bikes):
    result = []
    for bike in bikes:
        result.append({
            "lng": bike['longitude'],
            "lat": bike['latitude']
        })
    return result

data = {
    "bikes": latLng(bikes),
    "points": list(map(lambda p: p._asdict(), POINTS))
}

with Path('bikes.json').open('w') as file:
    file.write(json.dumps(data, indent=2, sort_keys=True))